In [ ]:
#importing packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/big-mart-sales/Train-Set.csv")
test = pd.read_csv("/kaggle/input/big-mart-sales/Test-Set.csv")

## Getting the overview

In [ ]:
# getting the first 5 rows

train.head()

In [ ]:
# shape of the data

train.shape

In [ ]:
# getting the overview 

train.info()

In [ ]:
# Get categorical columns
categorical_columns = train.select_dtypes(include=['object', 'category']).columns.tolist()

# Get numerical columns
numerical_columns = train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical Columns:", categorical_columns)
print("Numerical Columns:", numerical_columns)

In [ ]:
# getting the percentage of null data

train.isnull().mean()*100

## Handeling the null values